<h1><center>Data Preprocessing - BART Base</center></h1>

In [3]:
# import needed libraries and checkpoint models 
import json
import tensorflow as tf
from transformers import BartTokenizer, TFBartModel

# initalize the tokenizer and model for embeddings 
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = TFBartModel.from_pretrained('facebook/bart-base')


# Load data from JSON file
with open('../data/nytfox_collate_content_quick_v1.json', 'r') as f:
    articles = json.load(f)

2023-04-19 19:46:02.337449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 19:46:15.263553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBartModel.

All the layers of TFBartModel were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartModel for predictions without further traini

In [4]:
# Load NYT data from JSON file
with open('../data/nyt_collate_content_v1.json', 'r') as f:
    nyt_articles = json.load(f)

In [21]:
# mandate the maximum size of the text and title embeddings 
max_input_length = 512
max_target_length = 30

# function to embed words from the articles and titles 
def embed_data(article):
    
    # dictionary to save tokenized article
    article_tokens = {}
    
    # tokenize the contents of an article 
    content_tokens = tokenizer(
        article["content"],
        max_length=max_input_length,
        truncation=True,
    )
    
    # save tokenized content and masks to dict
    article_tokens['content'] = content_tokens['input_ids']
    article_tokens['content_mask'] = content_tokens['attention_mask']
    
    # tokenize the title of an article
    title_tokens = tokenizer(
        article["title"], max_length=max_target_length, truncation=True
    )
    
    # save tokenized title and masks to dict
    article_tokens["title"] = title_tokens["input_ids"]
    article_tokens['title_mask'] = title_tokens['attention_mask']
    
    
    outputs = model(article_tokens['content'])
    last_hidden_states = outputs.last_hidden_state
    
    print(last_hidden_states)
    return article_tokens

In [25]:
import tensorflow as tf
from transformers import TFBartModel, BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = TFBartModel.from_pretrained('facebook/bart-base')

def embed_data(article):
    article_tokens = {}

    # tokenize the contents of an article
    content_tokens = tokenizer(
        article["content"],
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

    # save tokenized content and masks to dict
    article_tokens['content'] = content_tokens['input_ids']
    article_tokens['content_mask'] = content_tokens['attention_mask']

    # tokenize the title of an article
    title_tokens = tokenizer(
        article["title"],
        max_length=max_target_length,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

    # save tokenized title and masks to dict
    article_tokens["title"] = title_tokens["input_ids"]
    article_tokens['title_mask'] = title_tokens['attention_mask']

    # get the embeddings
    with tf.device('/cpu:0'):
        content_embed = model(content_tokens['input_ids'], attention_mask=content_tokens['attention_mask'])[0]
        title_embed = model(title_tokens['input_ids'], attention_mask=title_tokens['attention_mask'])[0]

    # save the embeddings to dict
    article_tokens['content_embed'] = content_embed
    article_tokens['title_embed'] = title_embed

    return article_tokens



All model checkpoint layers were used when initializing TFBartModel.

All the layers of TFBartModel were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartModel for predictions without further training.


In [26]:
# list to save embedded articles 
embedded_articles = []

# embed each article and title 
for article in articles[0:1]:
    
    # create tokenized dictionary 
    article_tokens = embed_data(article)
    
    # save to master list 
    embedded_articles.append(article_tokens)

In [27]:
embedded_articles

[{'content': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
  array([[    0, 39679,  4014,  5121,  7481,    35,    83,  3782,     9,
            753,  3354,  8973,  1134,  5417,   296,    71,     5,   446,
           1595,     5,  9978,  2169, 32125,  1783,     8,   373,    15,
              5,  1112,     7, 21790, 11511,     5,  2309,     4, 50118,
          50118,   133,  1134,     6,   669,    30,  1791,    13, 30052,
           1571,     6, 17407,     5,  1087,    25,    22,  1245,  6920,
           2309,    60,  7594,    24,    74,  3231,   270, 15478,    18,
           2147,  4026,     8,  9128,  3575,  2680,     4,    20,  2309,
           1171,  7668,     7,  2501,  1897,  1007,   931,     6,  8373,
             62,  2008, 10026,  1787,  9781,     8,  3114,  2210, 19289,
           2074,     4, 50118, 50118,   113,   170, 19781,     5,   446,
             13,  3133,    42, 10043,  2309,     7, 26845, 11685,  1007,
          18225,    60,     5,  1134,    26,    11,    10,   44

In [8]:
# list to save embedded articles 
embedded_articles = []

# embed each article and title 
for article in nyt_articles:
    
    # create tokenized dictionary 
    article_tokens = embed_data(article)
    
    # save to master list 
    embedded_articles.append(article_tokens)

In [52]:
tokenizer.convert_ids_to_tokens(embedded_articles[1]['title'])

['<s>',
 'B',
 'iden',
 'Ġnominee',
 'Ġcoordinated',
 'Ġdark',
 'Ġmoney',
 'Ġclimate',
 'Ġnuisance',
 'Ġlawsuits',
 'Ġinvolving',
 'ĠLeonardo',
 'ĠDi',
 'Cap',
 'rio',
 '</s>']